In [1]:
import random
import numpy as np 
import pandas as pd
import operator
from scipy.sparse import coo_matrix
from numpy.linalg import norm
from sklearn.metrics import mean_squared_error

In [2]:
ls

 C ����̺��� ����: Windows
 ���� �Ϸ� ��ȣ: C211-4D3F

 c:\Users\songbae\BoostCamp\Day23 ���͸�

2021-02-24  ���� 02:51    <DIR>          .
2021-02-24  ���� 02:51    <DIR>          ..
2021-02-24  ���� 05:32            38,205 2nd_assignment.ipynb
2021-02-24  ���� 11:13             5,766 Community_Girvan_Newman.ipynb
2021-02-24  ���� 12:20             5,065 graph_in_recommend.ipynb
               3�� ����              49,036 ����Ʈ
               2�� ���͸�  329,857,384,448 ����Ʈ ����


In [3]:
# import os, sys 
# from google.colab import drive 

# ### 해당 코드 실행 시 colab에서 실행중인 폴더의 /content/drive/My Drive가 구글 드라이브에 연결됨

# drive.mount('/content/drive')


In [4]:
#### 영화 데이터셋 형태 확인 ####
path='../data/others/movies.csv'
movies = pd.read_csv(path)
print("### Movie Dataset Format ###", end = '\n\n')
movies.head()

### Movie Dataset Format ###



,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
################### 가상 유저 추가해주기 위한 작업 데이터 전처리 - 장르별 영화 리스트를 뽑는다 #################

# 이 셀을 실행하는 이유는 후에 특정 장르를 좋아하는 가상의 유저를 만들어서 추천의 결과를 보기 위함입니다.

movie_dict = dict()                 # {movie_id : (movie_title, movie_genre)}
musical_list = list()               # musical 장르의 영화들
horror_list = list()                # horror 장르의 영화들
documentary_list = list()           # documentary 장르의 영화들
comedy_list = list()                # comedy 장르의 영화들
animation_list = list()

for (movie_id, movie_title, movie_genre) in movies.itertuples(index=False):
    movie_dict[movie_id] = (movie_title, movie_genre)
    if 'Musical' in movie_genre:
        musical_list.append(movie_id)
    if 'Horror' in movie_genre:
        horror_list.append(movie_id)
    if 'Documentary' in movie_genre:
        documentary_list.append(movie_id)
    if 'Comedy' in movie_genre:
        comedy_list.append(movie_id)
    if 'Animation' in movie_genre:
        animation_list.append(movie_id)
# print(musical_list)
# print(horror_list)
# print(documentary_list)
# print(comedy_list)

In [6]:
path='../data/others/ratings.csv'
ratings = pd.read_csv(path)

# 평점 데이터셋 형태 확인
print("### Rating Dataset Format ###", end='\n\n')
print(ratings.head(), end='\n\n\n')
ratings.drop(['timestamp'], axis=1, inplace=True)
print(ratings.head())
print(ratings.info())
100836//len(ratings['userId'].unique())

### Rating Dataset Format ###

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931


   userId  movieId  rating
0       1        1     4.0
1       1        3     4.0
2       1        6     4.0
3       1       47     5.0
4       1       50     5.0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100836 non-null  int64  
 1   movieId  100836 non-null  int64  
 2   rating   100836 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB
None


165

In [7]:
'''
    유저가 평균적으로 영화를 130개정도 본 것으로 나타나서 일반적으로 추가하는 사람들도 그정도로 설정
    그리고, 영화에 대한 평점을 줄 때는 영화 순서를 섞고, 무조건 5점을 주는 것이 아니라 4, 5점 중 랜덤으로 설정
    낮은 점수를 줄 때도 마찬가지.
'''
# uid==800을 가지는 유저를 추가한다. 이 유저는 뮤지컬의 광팬이다.

random.shuffle(musical_list);
random.shuffle(horror_list);
random.shuffle(documentary_list);
random.shuffle(comedy_list);

print(ratings)
new_uid = 800
rows = list()
for movie_id in musical_list[:100]:
    rows.append([new_uid, movie_id, random.randint(4,5)])
for movie_id in horror_list[:50]:
    rows.append([new_uid, movie_id, random.randint(1,2)])
for movie_id in documentary_list[:20]:
    rows.append([new_uid, movie_id, random.randint(2,3)])
for row in rows:
    ratings = ratings.append(pd.Series(row, index=ratings.columns), ignore_index=True)
print(ratings)


        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
100831     610   166534     4.0
100832     610   168248     5.0
100833     610   168250     5.0
100834     610   168252     5.0
100835     610   170875     3.0

[100836 rows x 3 columns]
        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
101001     800    68522     2.0
101002     800    54881     2.0
101003     800    76751     3.0
101004     800   171251     2.0
101005     800     8577     3.0

[101006 rows x 3 columns]


In [8]:
# uid==850을 가지는 유저를 추가한다. 이 유저는 다큐 영화의 광팬이다.

random.shuffle(musical_list);
random.shuffle(horror_list);
random.shuffle(documentary_list);
random.shuffle(comedy_list);
random.shuffle(animation_list);


print(ratings)
new_uid = 850
rows = list()
for movie_id in documentary_list[:100]:
    rows.append([new_uid, movie_id, random.randint(4,5)])
for movie_id in horror_list[:50]:
    rows.append([new_uid, movie_id, random.randint(1,2)])
for movie_id in animation_list[:10]:
    rows.append([new_uid, movie_id, random.randint(1,2)])

for row in rows:
    ratings = ratings.append(pd.Series(row, index=ratings.columns), ignore_index=True)
print(ratings)

        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
101001     800    68522     2.0
101002     800    54881     2.0
101003     800    76751     3.0
101004     800   171251     2.0
101005     800     8577     3.0

[101006 rows x 3 columns]
        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
101161     850     1920     1.0
101162     850     2102     2.0
101163     850     8574     2.0
101164     850    55768     1.0
101165     850    31658     1.0

[101166 rows x 3 columns]


In [9]:
# uid==900을 가지는 유저를 추가한다. 이 유저는 호러의 광팬이다.

random.shuffle(musical_list);
random.shuffle(horror_list);
random.shuffle(documentary_list);
random.shuffle(comedy_list);
random.shuffle(animation_list);

new_uid = 900
rows = list()
for movie_id in horror_list[:120]:
    rows.append([new_uid, movie_id, random.randint(4,5)])
for movie_id in documentary_list[:10]:
    rows.append([new_uid, movie_id, random.randint(1,2)])
for movie_id in comedy_list[:30]:
    rows.append([new_uid, movie_id, random.randint(1,2)])
for movie_id in animation_list[:20]:
    rows.append([new_uid, movie_id, random.randint(1,2)])
for row in rows:
    ratings = ratings.append(pd.Series(row, index=ratings.columns), ignore_index=True)
print(ratings)

        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
101341     900    95149     1.0
101342     900     5618     2.0
101343     900   108090     2.0
101344     900   170957     1.0
101345     900   107999     1.0

[101346 rows x 3 columns]


In [10]:
# uid==950을 가지는 유저를 추가한다. 이 유저는 애니메이션의 광팬이다.

random.shuffle(musical_list);
random.shuffle(horror_list);
random.shuffle(documentary_list);
random.shuffle(comedy_list);
random.shuffle(animation_list);

new_uid = 950
rows = list()
for movie_id in horror_list[:20]:
    rows.append([new_uid, movie_id, random.randint(1,2)])
for movie_id in documentary_list[:10]:
    rows.append([new_uid, movie_id, random.randint(1,3)])
for movie_id in comedy_list[:30]:
    rows.append([new_uid, movie_id, random.randint(2,4)])
for movie_id in animation_list[:150]:
    rows.append([new_uid, movie_id, random.randint(3,5)])
for row in rows:
    ratings = ratings.append(pd.Series(row, index=ratings.columns), ignore_index=True)
print(ratings)

        userId  movieId  rating
0            1        1     4.0
1            1        3     4.0
2            1        6     4.0
3            1       47     5.0
4            1       50     5.0
...        ...      ...     ...
101551     950      673     3.0
101552     950    91355     4.0
101553     950    71899     3.0
101554     950    33090     4.0
101555     950     1274     3.0

[101556 rows x 3 columns]


In [11]:
########################################################################
######### Mapping user id, movie id to user index, movie index #########
########################################################################

uid_2_idx = dict()     # user id --> user idx
mid_2_idx = dict()    # movie id --> movie idx

u_idx = 0
m_idx = 0
for user_id, movie_id, r in ratings.itertuples(index=False):
    if user_id not in uid_2_idx:
        uid_2_idx[user_id] = u_idx
        u_idx = u_idx+1

    if movie_id not in mid_2_idx:
        mid_2_idx[movie_id] = m_idx
        m_idx = m_idx+1

num_user = len(uid_2_idx)
num_movie = len(mid_2_idx)
print(num_user) # 유저수
print(num_movie) # 영화수

uidx_2_id = {v:k for (k,v) in uid_2_idx.items()}    # user idx --> user id
midx_2_id = {v:k for (k,v) in mid_2_idx.items()}    # movie idx --> movie id


614
9725


In [12]:
# rating matrix 초기화
rating_matrix = np.zeros((num_user, num_movie)) # num_user x num_movie를 크기로 가지는 numpy 행렬 선언, 각 요소는 유저가 영화에 매긴 rating을 나타냄.

for user_id, movie_id, r in ratings.itertuples(index=False):
    u_idx = uid_2_idx[user_id]
    m_idx = mid_2_idx[movie_id]
    rating_matrix[u_idx, m_idx] = r

rating_matrix

array([[4., 4., 4., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 3.]])

In [13]:
######################################################################################################################################
# Training Set과 Test Set을 분리해 주는 함수
# kind='user'인 경우 같은 user 내의 rating을, kind = 'item'인 경우 같은 item(movie) 내의 rating을 train set과 test set으로 분리해준다.
######################################################################################################################################
def train_test_split(ratings):
    test = np.zeros_like(ratings)
    train = ratings.copy()
    for x in range(ratings.shape[0]):
        nonzero_idx = ratings[x, :].nonzero()[0]
        test_ratings = np.random.choice(nonzero_idx, 
                                        size=int(len(nonzero_idx)/5),  
                                        replace=False)
        train[x, test_ratings] = 0.
        test[x, test_ratings] = ratings[x, test_ratings]
        
    assert(np.all((train * test) == 0))     # train set과 test set이 완전히 분리되었는지 확인

    return train, test

######################################################################################
###################### 유저별 평균을 기존 rating에서 빼준다 ##########################
######################################################################################
####### *****rating이 "있는" 값들의 평균만을 rating이 있는 곳에서만 빼준다############
def subtract_mean_ratings(ratings):
    mean_subtracted_ratings = np.zeros_like(ratings)
    avg_ratings = np.zeros(ratings.shape[0])
    for i in range(ratings.shape[0]):
        nonzero_idx = ratings[i].nonzero()[0]                       # rating 값이 존재하는(0이 아닌) index 추출
        sum_ratings = np.sum(ratings[i])          
        num_nonzero = len(nonzero_idx)
        avg_rating = sum_ratings / num_nonzero                      # rating 값들의 평균값 계산
        if num_nonzero == 0:                  
            avg_rating = 0
        avg_ratings[i] = avg_rating
        mean_subtracted_ratings[i, nonzero_idx] = ratings[i, nonzero_idx] - avg_rating 
                                                                    # 원 rating matrix에서 평균 값을 빼줌
    return mean_subtracted_ratings, avg_ratings

In [14]:
train_ratings, test_ratings = train_test_split(rating_matrix)
mean_subtracted_ratings, avg_ratings = subtract_mean_ratings(train_ratings)

In [48]:
def calculate_rmse(R, U, V, lambda_u, lambda_v):
   
    error = 0
    for u, i, r_ui in zip(R.row, R.col, R.data):
        error += (r_ui-U[u,:]@V[:,i])**2
    for u in range(U.shape[0]):
        error += lambda_u*(norm(U[u,:])**2)
    for i in range(V.shape[1]):
        error += lambda_v*(norm(V[:,i])**2)
    rmse = np.sqrt(error/len(R.data))
    return rmse

def SGD(U, V, R, lr, lambda_u, lambda_v):
    for u, i, r_ui in R:
        e=r_ui-U[u,:]@V[:,i]
        grad_u=2*e*(-V[:,i])+2*lambda_u*U[u,:]
        grad_i=2*e*(-U[u,:])+2*lambda_v*V[:,i]
        U[u,:]+=-lr*grad_u
        V[:,i]+=-lr*grad_i
    return U, V


def train(ratings, dim=10, max_epoch=70, lambda_u=0.1, lambda_v=0.1, lr=0.01):
    #lambda_u : regularization coefficient of U matrix, lambda_v : regularizatoin coeff of V matrix,
    # lr : learning rate
    num_u, num_i = ratings.shape

    U = np.random.rand(num_u, dim)
    V = np.random.rand(dim, num_i)
    R = coo_matrix(ratings)
    rmse = calculate_rmse(R, U, V, lambda_u, lambda_v)
    print("Initial RMSE: " + str(rmse))

    for epoch in range(max_epoch):
        if epoch != 0 and epoch%100==0:
            lr /= 2
        rating_data = list(zip(R.row, R.col, R.data))
        random.shuffle(rating_data)
        U, V = SGD(U, V, rating_data, lr, lambda_u, lambda_v)
        rmse = calculate_rmse(R, U, V, lambda_u, lambda_v)        
        print('Epoch: {:5}, RMSE: {:15}, Learning Rate:{}'.format(epoch, rmse, lr))
    
    return U, V 

def predict(U, V, user_id=None):
    # user_id가 주어지지 않은 경우엔 전체 예측된 평점 행렬을 return,
    # 주어진 경우엔 해당 유저에 대한 예측 평점 행렬을 return한다.

    if user_id is None:
        return np.dot(U, V)
    else:
        user_idx = uid_2_idx[user_id]
        return np.dot(U[user_idx], V )


#######################################################################################
##########user id = uid인 유저에게 유저가 본 영화를 제외하고 N개를 추천해준다######
#######################################################################################
def recommend_for_uid(uid, ori_rating_matrix, U, V, top_N):

    movie_rating_dict = dict()
    u_predicted_rating = predict(U, V, user_id=uid)
    for movie_idx, pred_rating in enumerate(u_predicted_rating):
        movie_rating_dict[movie_idx] = pred_rating
    
    uidx = uid_2_idx[uid]
    sorted_dict = sorted(movie_rating_dict.items(), key=operator.itemgetter(1), reverse=True)       # 전체 영화를 대상으로 예측 rating을 기준으로 나열
    already_seen_movie_idxs = np.nonzero(ori_rating_matrix[uidx])[0]                                # 유저가 평점을매긴(이미 본) 영화들의 idx

    print_cnt = 0
    for idx, pred_rating in sorted_dict:
        if print_cnt == top_N:
            break
        if idx not in already_seen_movie_idxs:
            print(str(movie_dict[midx_2_id[idx]])+"    "+str(pred_rating))
            print_cnt = print_cnt+1


In [49]:
U, V = train(mean_subtracted_ratings)

Initial RMSE: 2.7719156094155317
Epoch:     0, RMSE: 0.9804451773185788, Learning Rate:0.01
Epoch:     1, RMSE: 0.9531042585201651, Learning Rate:0.01
Epoch:     2, RMSE: 0.9453179896138676, Learning Rate:0.01
Epoch:     3, RMSE: 0.9363549905271078, Learning Rate:0.01
Epoch:     4, RMSE: 0.9256287164102178, Learning Rate:0.01
Epoch:     5, RMSE: 0.9117856666414316, Learning Rate:0.01
Epoch:     6, RMSE: 0.8908024677979066, Learning Rate:0.01
Epoch:     7, RMSE: 0.8730158012377554, Learning Rate:0.01
Epoch:     8, RMSE: 0.855142332207435, Learning Rate:0.01
Epoch:     9, RMSE: 0.8408894580930332, Learning Rate:0.01
Epoch:    10, RMSE: 0.828081667967556, Learning Rate:0.01
Epoch:    11, RMSE: 0.8166182729232132, Learning Rate:0.01
Epoch:    12, RMSE: 0.8054311575027244, Learning Rate:0.01
Epoch:    13, RMSE: 0.7964079858103561, Learning Rate:0.01
Epoch:    14, RMSE: 0.7863290003154818, Learning Rate:0.01
Epoch:    15, RMSE: 0.7790484616947486, Learning Rate:0.01
Epoch:    16, RMSE: 0.770

In [50]:
predicted_ratings = predict(U, V)
predicted_ratings += avg_ratings.reshape([-1,1])
print(predicted_ratings)

[[4.38432089 3.98795701 4.53243105 ... 4.49731607 4.48418146 4.09962283]
 [3.91616288 4.40251127 3.7293069  ... 3.91989291 3.83072861 4.59401518]
 [1.36406163 2.87349719 2.3384387  ... 2.07132923 3.4265934  2.79320133]
 ...
 [3.71461564 2.67503377 3.06330412 ... 3.85578392 2.88510472 4.19056383]
 [2.43505884 4.13268241 3.42757865 ... 3.30793677 4.34499606 2.69698968]
 [3.2879907  3.63007161 2.98754151 ... 3.04167478 2.87428927 3.95306823]]


In [51]:
# 타겟 유저를 정해서, 해당 유저가 본 영화들의 분포를 뽑아내보기
# 유저의 추첨 결과가 올바르게 나왔는지 해석하기 위하여 해당 함수를 만들어서 추천 결과와 출력합니다.  
def print_user_preference(ori_rating_matrix, u_idx, type):
    '''
        type : ['avg', 'sum', 'cnt']
        avg 는 평균 평점, sum은 평점의 합, cnt은 평점 매긴 횟수
    '''
    target_hist = ori_rating_matrix[u_idx]
    target_seen_movie_idx =  np.nonzero(target_hist)[0]
    target_genre_dict = dict()
    target_genre_sum_rating_dict = dict()
    for movie_idx in target_seen_movie_idx:
        _, genre = movie_dict[midx_2_id[movie_idx]]
        for g in genre.split("|"):
            if g not in target_genre_dict.keys():
                target_genre_dict[g]=0
                target_genre_sum_rating_dict[g]=0
            target_genre_dict[g]=target_genre_dict[g]+1
            target_genre_sum_rating_dict[g] = target_genre_sum_rating_dict[g] + ori_rating_matrix[u_idx][movie_idx]

    # # 많이 본 횟수 대로 
    if type=='cnt':
        target_genre_dict = sorted(target_genre_dict.items(), key=operator.itemgetter(1), reverse=True)
        for k,v in target_genre_dict:
            print(k,v)
    
    # # 영화 평점의 합대로
    elif type=='sum':
        target_genre_sum_rating_dict = sorted(target_genre_sum_rating_dict.items(), key=operator.itemgetter(1), reverse=True)
        for k,v in target_genre_sum_rating_dict:
            print(k,v)
        
    # 영화 평균 평점 순서대로
    elif type=="avg":    
        target_genre_avg_rating_dict = dict()
        for k,v in target_genre_sum_rating_dict.items():
            target_genre_avg_rating_dict[k] = v/target_genre_dict[k]
        target_genre_avg_rating_dict = sorted(target_genre_avg_rating_dict.items(), key=operator.itemgetter(1), reverse=True)
        for k,v in target_genre_avg_rating_dict:
            print(k,v)
    
    else:
        target_genre_avg_rating_dict = dict()
        for k,v in target_genre_sum_rating_dict.items():
            target_genre_avg_rating_dict[k] = v/target_genre_dict[k]
        target_genre_avg_rating_dict = sorted(target_genre_avg_rating_dict.items(), key=operator.itemgetter(1), reverse=True)
        for k,v in target_genre_avg_rating_dict:
            print(k,v,target_genre_dict[k],target_genre_sum_rating_dict[k])

In [52]:
##### Check the RMSE of the test set ####### 
pred = predicted_ratings[test_ratings.nonzero()].flatten()
actual = test_ratings[test_ratings.nonzero()].flatten()

print("### Test RMSE ###")
print(np.sqrt(mean_squared_error(pred,actual)))

### Test RMSE ###
0.868331088250249


In [53]:
musical_fan = 800
docu_fan = 850
horror_fan = 900
animation_fan =  950

In [54]:
# 해당 유저가 본 영화의 분포를 알고 싶으면 print_user_preference 함수를 실행시켜보세요!
# print_user_preference(rating_matrix, musical_fan, 'sum')
recommend_for_uid(musical_fan, rating_matrix, U, V, top_N=30)

('Skin I Live In, The (La piel que habito) (2011)', 'Drama')    2.8719864851368806
('Pokemon 4 Ever (a.k.a. Pokémon 4: The Movie) (2002)', 'Adventure|Animation|Children|Fantasy')    2.5909784769957573
('Meatballs (1979)', 'Comedy')    2.1667016456165826
('Muppets From Space (1999)', 'Children|Comedy')    2.0880548051021655
('Titanic (1953)', 'Action|Drama')    2.040160396277404
('Pollyanna (1960)', 'Children|Comedy|Drama')    1.9539634062096471
('Cage Dive (2017)', 'Drama|Horror|Thriller')    1.8392872240043294
('Lake House, The (2006)', 'Drama|Fantasy|Romance')    1.8190291236762484
('Tooth Fairy 2 (2012)', 'Children|Comedy')    1.7828244294361004
('Louis C.K.: Chewed Up (2008)', 'Comedy')    1.7605253109234336
('Circle of Friends (1995)', 'Drama|Romance')    1.7522477616999252
('Crow: City of Angels, The (1996)', 'Action|Thriller')    1.7520240226862913
('Misfits, The (1961)', 'Comedy|Drama|Romance|Western')    1.7462547016542498
('We Were Soldiers (2002)', 'Action|Drama|War')    1.7

In [55]:
# 해당 유저가 본 영화의 분포를 알고 싶으면 print_user_preference 함수를 실행시켜보세요!
# print_user_preference(rating_matrix, docu_fan,'sum')
recommend_for_uid(docu_fan, rating_matrix, U, V, top_N=30)

('Holy Mountain, The (Montaña sagrada, La) (1973)', 'Drama')    1.9112088464526855
('10 Cent Pistol (2015)', 'Crime|Thriller')    1.902058540011947
('Tender Mercies (1983)', 'Drama|Romance|Western')    1.8382627845942023
('Films to Keep You Awake: The Christmas Tale (Películas para no dormir: Cuento de navidad) (2005)', 'Horror|Thriller')    1.8299349402086706
('Next Best Thing, The (2000)', 'Comedy|Drama')    1.7760230033349975
('Fantasia (1940)', 'Animation|Children|Fantasy|Musical')    1.6908718846140711
('Mrs. Henderson Presents (2005)', 'Comedy|Drama')    1.6709597863177574
('Grey Gardens (1975)', 'Documentary')    1.6693666509104663
('Another Woman (1988)', 'Drama')    1.6400728331005299
('Gallipoli (1981)', 'Drama|War')    1.6300929218766145
('Alex and Emma (2003)', 'Comedy|Drama|Romance')    1.5805336948949091
('Akira (1988)', 'Action|Adventure|Animation|Sci-Fi')    1.5763933176320242
('Suspiria (1977)', 'Horror')    1.5666033823964023
('Happiness of the Katakuris, The (Katakur

In [56]:
# 해당 유저가 본 영화의 분포를 알고 싶으면 print_user_preference 함수를 실행시켜보세요!
# print_user_preference(rating_matrix, horror_fan,'sum')
recommend_for_uid(horror_fan, rating_matrix, U, V, top_N=30)

('Happy Feet (2006)', 'Adventure|Animation|Children|Comedy|IMAX')    2.1220638210852965
('House of Mirth, The (2000)', 'Romance')    2.025772568993334
('Rogue (2007)', 'Action|Adventure|Horror|Sci-Fi|Thriller')    2.021448217836149
('R.I.P.D. (2013)', 'Action|Comedy|Fantasy')    1.9474902151212647
('I Still Know What You Did Last Summer (1998)', 'Horror|Mystery|Thriller')    1.931205157890444
('Candyman (1992)', 'Horror|Thriller')    1.8856868180451318
("Eight Crazy Nights (Adam Sandler's Eight Crazy Nights) (2002)", 'Animation|Comedy|Musical')    1.8731321491804673
('Saw (2004)', 'Horror|Mystery|Thriller')    1.870313062239794
('Spy Kids 3-D: Game Over (2003)', 'Action|Adventure|Children')    1.8532377981100656
('Dr. Jekyll and Mr. Hyde (1931)', 'Drama|Horror')    1.843254542570478
('Mr. Wrong (1996)', 'Comedy')    1.8317676237154314
('Bad Milo (Bad Milo!) (2013)', 'Comedy|Horror')    1.793848777081569
('Alice Through the Looking Glass (2016)', 'Adventure|Children|Fantasy')    1.78101

In [57]:
# 해당 유저가 본 영화의 분포를 알고 싶으면 print_user_preference 함수를 실행시켜보세요!
# print_user_preference(rating_matrix, horror_fan,'sum')
recommend_for_uid(animation_fan, rating_matrix, U, V, top_N=30)

('Ted 2 (2015)', 'Comedy')    2.033394478789866
('The Interview (2014)', 'Action|Comedy')    1.6420637287505941
('Claymation Christmas Celebration, A (1987)', 'Animation|Children|Comedy|Musical')    1.6393714989518053
('Christmas Carol, A (1977)', 'Drama|Fantasy')    1.6004374997867388
("It's a Very Merry Muppet Christmas Movie (2002)", 'Children|Comedy')    1.5570993795472243
('The Gracefield Incident (2015)', 'Action|Horror|Sci-Fi')    1.5087168201254446
('The Purge: Election Year (2016)', 'Action|Horror|Sci-Fi')    1.4985274512339883
('I Origins (2014)', 'Drama|Sci-Fi')    1.472415937187882
('Wonder (2017)', 'Drama')    1.432137482057697
('Satanic (2016)', 'Horror')    1.4054204855491146
('Toxic Avenger Part III: The Last Temptation of Toxie, The (1989)', 'Comedy|Horror')    1.397125723333777
('High and Low (Tengoku to jigoku) (1963)', 'Crime|Drama|Film-Noir|Thriller')    1.3830992752912856
('Amer (2009)', 'Drama|Horror')    1.3778815155090411
('Mamma Mia: Here We Go Again! (2018)',

# Test Code

In [58]:
################ Generating Synthetic Data #######################
synthetic_rating = np.zeros((5,10))

for i in range(10):
    random.seed(i)
    u_idx = random.randint(0,4)
    i_idx = random.randint(0,9)
    r_ui = random.randint(1,5)
    synthetic_rating[u_idx ,i_idx] = r_ui

synthetic_R = coo_matrix(synthetic_rating)
synthetic_R_zipped = list(zip(synthetic_R.row, synthetic_R.col, synthetic_R.data))
np.random.seed(7)
synthetic_U = np.random.rand(5, 3)
synthetic_V = np.random.rand(3, 10)

In [59]:
path='../data/others/'
answer_U = np.load(os.path.join(path,'answer_U.npy'))
answer_V = np.load(os.path.join(path,'answer_V.npy'))
answer_rmse = np.load(os.path.join(path,'answer_rmse.npy'))

In [60]:
def test_code(sgd, rmse):
    U, V = sgd(synthetic_U, synthetic_V, synthetic_R_zipped, lr=0.01, lambda_u=0.1, lambda_v=0.1) 
    mse_u = mean_squared_error(U, answer_U)
    mse_v = mean_squared_error(V, answer_V)
    print(mse_u,mse_v)
    assert(mse_u <3e-2 or mse_v < 1e-2), 'calculated U, V is different with the answer : SGD 함수 오류'   ## 정답과 오차범위 내의 값이 아니면 assertion 오류

    rmse = calculate_rmse(synthetic_R, U, V, lambda_u=0.1, lambda_v=0.1)
    print(rmse)
    assert(abs(answer_rmse-rmse)<1e-05), 'calculated rmse is diferent with the answer : RMSE 함수 오류'   ## 정답과 오차범위 내의 값이 아니면 assertion 오류
    
    print("모든 함수 알맞게 구현됨")


In [61]:
test_code(SGD, calculate_rmse)

0.0 0.0
2.5343996842764556


AssertionError: calculated rmse is diferent with the answer : RMSE 함수 오류